# Evaluation Study

## Setup

In [1]:
import os, sys
import shutil
import math
import time
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F

sys.path.append('..')

In [27]:
# PATH Setting
DIR_PATH = 'F:/20210730_samples'
SAVE_PATH = os.path.join(DIR_PATH, 'numpy')
CT_PATH = os.path.join(SAVE_PATH, 'CT_target')
PT_PATH = os.path.join(SAVE_PATH, 'PT_vanilla')
PT_PATH_2 = os.path.join(SAVE_PATH, 'PT_resize')

In [3]:
device = torch.device('cuda')

## NCC

In [29]:
from utils.common.metrics import NCC

Ii = np.zeros([16, 1, 128, 128, 256])
Ji = np.zeros([16, 1, 128, 128, 256])
for i in range(16):
    Ii[i,0] = np.load(os.path.join(CT_PATH, os.listdir(CT_PATH)[i]))
    Ji[i,0] = np.load(os.path.join(PT_PATH, os.listdir(PT_PATH)[i]))

Ii = torch.FloatTensor(Ii).to(device)
Ji = torch.FloatTensor(Ji).to(device)

criterion = NCC(device)
cc = criterion(Ii, Ji).cpu()
print ('Autograd Check', cc.requires_grad)
print ('NCC', cc)
print ('Average NCC', cc.mean().item())

Autograd Check False
NCC tensor([0.2473, 0.2616, 0.3115, 0.2672, 0.2100, 0.2890, 0.2440, 0.1606, 0.1522,
        0.1706, 0.2456, 0.1929, 0.2322, 0.1960, 0.3376, 0.2225])
Average NCC 0.23380093276500702


In [30]:
Ii = np.zeros([16, 1, 128, 128, 256])
Ji = np.zeros([16, 1, 128, 128, 256])
for i in range(16):
    Ii[i,0] = np.load(os.path.join(CT_PATH, os.listdir(CT_PATH)[i]))
    Ji[i,0] = np.load(os.path.join(PT_PATH_2, os.listdir(PT_PATH_2)[i]))

Ii = torch.FloatTensor(Ii).to(device)
Ji = torch.FloatTensor(Ji).to(device)

criterion = NCC(device)
cc = criterion(Ii, Ji).cpu()
print ('Autograd Check', cc.requires_grad)
print ('NCC', cc)
print ('Average NCC', cc.mean().item())

Autograd Check False
NCC tensor([0.4268, 0.4540, 0.5476, 0.4465, 0.3494, 0.5016, 0.3968, 0.2531, 0.2562,
        0.2873, 0.4179, 0.3197, 0.3976, 0.3193, 0.5823, 0.3773])
Average NCC 0.3958446681499481
